In [6]:
from collections import defaultdict, deque

# -----------------------------
# Parse netlist from file
# -----------------------------
def parse_netlist(filename):
    components = {}
    netlist = defaultdict(list)

    with open(filename, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith("COMPONENT"):
                parts = line.split()
                name = parts[1]
                delay = float(parts[2])
                comp_type = "LOGIC"
                setup = hold = 0
                for part in parts[3:]:
                    if "TYPE=" in part:
                        comp_type = part.split('=')[1]
                    elif "SETUP=" in part:
                        setup = float(part.split('=')[1])
                    elif "HOLD=" in part:
                        hold = float(part.split('=')[1])
                components[name] = {
                    'delay': delay,
                    'type': comp_type,
                    'setup': setup,
                    'hold': hold
                }
            elif line.startswith("CONNECTION"):
                _, src, dst = line.split()
                netlist[src].append(dst)

    return components, netlist

# -----------------------------
# Find FF-to-FF paths
# -----------------------------
def find_ff_to_ff_paths(components, netlist):
    ff_nodes = [name for name, comp in components.items() if comp['type'] == 'FF']
    ff_to_ff_paths = []

    for src_ff in ff_nodes:
        queue = deque([(src_ff, [src_ff])])
        while queue:
            current, path = queue.popleft()
            for neighbor in netlist.get(current, []):
                if components[neighbor]['type'] == 'LOGIC':
                    queue.append((neighbor, path + [neighbor]))
                elif components[neighbor]['type'] == 'FF' and neighbor != src_ff:
                    ff_to_ff_paths.append((src_ff, neighbor, path + [neighbor]))
    return ff_to_ff_paths

# -----------------------------
# Compute delay of a path
# -----------------------------
def compute_path_delay(path, components):
    delay = 0
    for node in path[1:-1]:  # exclude FFs at both ends
        delay += components[node]['delay']
    return delay

# -----------------------------
# Perform setup and hold analysis
# -----------------------------
def analyze_setup_hold(components, netlist, clock_period, skew):
    paths = find_ff_to_ff_paths(components, netlist)
    worst_arrival_plus_setup = 0
    critical_path_info = None

    for src, dst, path in paths:
        tclk_to_q = components[src]['delay']
        tcomb = compute_path_delay(path, components)
        tsetup = components[dst]['setup']
        thold = components[dst]['hold']

        arrival_time = tclk_to_q + tcomb

        # Setup and Hold Checks (not printed, just for completeness)
        setup_ok = (arrival_time + tsetup) <= (clock_period + skew)
        hold_ok = arrival_time >= (thold - skew)

        if arrival_time + tsetup > worst_arrival_plus_setup:
            worst_arrival_plus_setup = arrival_time + tsetup
            critical_path_info = {
                'path': path,
                'src': src,
                'dst': dst,
                'arrival_time': arrival_time,
                'tclk_to_q': tclk_to_q,
                'tcomb': tcomb,
                'tsetup': tsetup,
                'thold': thold
            }

    return worst_arrival_plus_setup, critical_path_info

# -----------------------------
# Main
# -----------------------------
filename = 'netlist2.txt'  # Change to your file
clock_period = 10.0        # Clock period in ns
skew = 6.0                 # Clock skew in ns

components, netlist = parse_netlist(filename)
worst_arrival_plus_setup, critical_path_info = analyze_setup_hold(components, netlist, clock_period, skew)

# -----------------------------
# Highlight Critical Path Only
# -----------------------------
print("\n" + "="*40)
print("Critical Path (Maximum Delay)")
print("="*40)
if critical_path_info:
    print("\n")
    print(f"Path: {' -> '.join(critical_path_info['path'])}")
    print(f"  Clock-to-Q (FF1): {critical_path_info['tclk_to_q']} ns")
    print(f"  Combinational Delay: {critical_path_info['tcomb']} ns")
    print(f"  Setup Time (FF2): {critical_path_info['tsetup']} ns")
    print(f"  Total Delay (Clk-Q + Comb + Setup): {critical_path_info['arrival_time'] + critical_path_info['tsetup']:.2f} ns")
    print(f"  Skew: {skew} ns")
else:
    print("No valid FF-to-FF path found.")

# -----------------------------
# Max frequency calculation
# -----------------------------
effective_critical_path = worst_arrival_plus_setup - skew
if effective_critical_path <= 0:
    print("\nInvalid configuration: Effective path delay <= 0. Cannot compute frequency.")
else:
    max_freq = 1 / (effective_critical_path * 1e-9)  # ns to s
    print("\n" + "="*40)
    print("Maximum Possible Frequency")
    print("="*40)
    print(f"\nCurrent Time period = {clock_period:.2f} ns")
    print(f"Minimum Time period required = {effective_critical_path:.2f} ns")
    print(f"Maximum Possible Frequency = {max_freq/1e6:.2f} MHz")
    print("\n")



Critical Path (Maximum Delay)


Path: DFF1 -> AND1 -> AND2 -> AND3 -> DFF2
  Clock-to-Q (FF1): 3.0 ns
  Combinational Delay: 11.0 ns
  Setup Time (FF2): 4.0 ns
  Total Delay (Clk-Q + Comb + Setup): 18.00 ns
  Skew: 6.0 ns

Maximum Possible Frequency

Current Time period = 10.00 ns
Minimum Time period required = 12.00 ns
Maximum Possible Frequency = 83.33 MHz


